In [1]:
from jupyter_set_up import init_django

init_django('pictionary')

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

In [3]:
class CNN(nn.Module):
    def __init__(self, classes):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 64, 3, 1, 1)
        self.conv2 = nn.Conv2d(64, 256, 3, 1, 1)
        self.conv3 = nn.Conv2d(256, 512, 3, 1, 1)

        self.fc1 = nn.Linear(512*3*3, 512)
        self.fc2 = nn.Linear(512, classes)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [4]:
def create_X_y(file):
    X = []
    y = []
    for label, img in np.load(file, allow_pickle=True):
        y.append(torch.tensor(int(label)-1))
        X.append(torch.tensor(img))
    return torch.stack(X), torch.stack(y)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train(cnn, train_X, train_y):
    BATCH_SIZE = 500
    EPOCHS = 30
    optimizer = optim.SGD(cnn.parameters(), lr=0.1)
    loss_function = nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
            
            batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 28, 28).to(device)
            batch_y = train_y[i:i+BATCH_SIZE].to(device)

            outputs = cnn(batch_X)
            loss = loss_function(outputs, batch_y)
            
            cnn.zero_grad()
            loss.backward()
            optimizer.step()
            
        print("Epoch: {} Loss: {}".format(epoch+1, loss))


def test(cnn, test_X, test_y):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(len(test_X))):
            real_class = test_y[i].to(device)
            net_out = cnn(test_X[i].view(-1, 1, 28, 28).to(device))[0]
            predicted_class = torch.argmax(net_out)
            if predicted_class == real_class:
                correct += 1
            total += 1
    print("Accuracy: {}".format(round(correct / total, 3)))

In [6]:
train_X, train_y = create_X_y('train.npy')

In [7]:
from PIL import Image

for i in range(-299, -1):
    img = Image.fromarray(np.array(train_X[i]))
    train_X[i] = torch.tensor(np.pad(np.array(img.resize(size=(26, 26))), 1, mode='constant'))

In [12]:
cnn1 = CNN(50).to(device)
train(cnn1, train_X, train_y)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3734/3734 [09:18<00:00,  6.69it/s]


Epoch: 1 Loss: 3.5012128353118896


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3734/3734 [09:28<00:00,  6.57it/s]


Epoch: 2 Loss: 2.9650461673736572


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3734/3734 [09:20<00:00,  6.67it/s]


...

Epoch: 28 Loss: 1.6229976415634155


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3734/3734 [09:19<00:00,  6.67it/s]


Epoch: 29 Loss: 1.6042802333831787


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3734/3734 [09:17<00:00,  6.70it/s]


Epoch: 30 Loss: 1.5898898839950562


In [8]:
test_X, test_y = create_X_y('test.npy')

In [11]:
for i in range(-149, -1):
    img = Image.fromarray(np.array(test_X[i]))
    test_X[i] = torch.tensor(np.pad(np.array(img.resize(size=(26, 26))), 1, mode='constant'))

In [12]:
test(cnn1, test_X, test_y)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 933509/933509 [21:14<00:00, 732.73it/s]

Accuracy: 0.861


In [21]:
import pickle
pickle.dump(cnn1,open("cnn_model.sav", "wb"))

In [30]:
model = pickle.load(open("cnn_model.sav", "rb"))
output = model(test_X[-7].view(-1, 1, 28, 28).to(device))[0]
prediction = torch.argmax(output)
int(prediction), int(test_y[-7])

(46, 48)